# Aplicando ETL a las ventas de Spotify

**Importamos las librerias**

In [2]:
import pandas as pd
import seaborn as sns
import sqlalchemy
import psycopg2

# Parametro de conexion
server = '127.0.0.1'
db = 'BD_ETL_Spotify'
user = 'postgres'
passw = '1234'
port = '5432'
ft = 'spotifydataset'

#Creamos el motor
engine = sqlalchemy.create_engine('postgresql+psycopg2://'+user+':'+passw+'@'+server+':'+port+'/'+db+'')

**Cargamos el DataFrame**

In [3]:
df = pd.read_csv("T:/Archivo Jupyter/DataFrames/spotifydataset.csv")


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1000 non-null   int64  
 1   artist_name        1000 non-null   object 
 2   genres             837 non-null    object 
 3   followers          1000 non-null   int64  
 4   artist_popularity  1000 non-null   int64  
 5   artist_url         1000 non-null   object 
 6   track_name         1000 non-null   object 
 7   album_name         1000 non-null   object 
 8   release_date       1000 non-null   object 
 9   duration_ms        1000 non-null   int64  
 10  explicit           1000 non-null   bool   
 11  track_popularity   1000 non-null   int64  
 12  danceability       1000 non-null   float64
 13  energy             1000 non-null   float64
 14  key                1000 non-null   int64  
 15  loudness           1000 non-null   float64
 16  mode               1000 n

In [101]:
df.dtypes


Unnamed: 0             int64
artist_name           object
genres                object
followers              int64
artist_popularity      int64
artist_url            object
track_name            object
album_name            object
release_date          object
duration_ms            int64
explicit                bool
track_popularity       int64
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
dtype: object

In [5]:
df.describe() 

,Unnamed: 0,followers,artist_popularity,duration_ms,track_popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,1000.000000,1.000000e+03,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,499.500000,1.150655e+07,59.429000,212368.619000,59.429000,0.611430,0.660562,5.243000,-7.160373,0.613000,0.091402,0.265840,0.074700,0.188810,0.516641,122.143448
std,288.819436,2.293890e+07,23.884963,65706.472572,23.884963,0.167246,0.207568,3.613725,4.479642,0.487307,0.090441,0.288895,0.225472,0.145454,0.232896,28.053658
min,0.000000,2.000000e+00,0.000000,51680.000000,0.000000,0.069000,0.003540,0.000000,-39.482000,0.000000,0.023200,0.000005,0.000000,0.026000,0.032600,49.305000
25%,249.750000,1.038784e+06,57.000000,173083.000000,57.000000,0.505500,0.539500,2.000000,-8.334250,0.000000,0.037975,0.028325,0.000000,0.095700,0.343500,99.983500
50%,499.500000,3.923159e+06,67.000000,204382.500000,67.000000,0.634000,0.680500,5.000000,-5.943500,1.000000,0.054800,0.152500,0.000004,0.129000,0.522000,122.073500
75%,749.250000,1.142132e+07,74.000000,244192.750000,74.000000,0.728250,0.820000,9.000000,-4.582500,1.000000,0.106000,0.416250,0.000586,0.246500,0.686250,138.667250
max,999.000000,1.194287e+08,92.000000,707160.000000,92.000000,0.964000,0.998000,11.000000,0.273000,1.000000,0.874000,0.996000,0.972000,0.984000,0.973000,201.776000


In [6]:
# Eliminamos ese Unnamed y la URl del artista para tener mas limpio el dataset
df = df.drop(columns = ['Unnamed: 0','artist_url'])
df

,artist_name,genres,followers,artist_popularity,track_name,album_name,release_date,duration_ms,explicit,track_popularity,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Ariana Grande,pop,98934105,89,we can't be friends (wait for your love),eternal sunshine,08/03/2024,228639,False,89,...,0.646,5,-8.334,1,0.0427,0.0615,0.000030,0.0740,0.295,115.842
1,Ariana Grande,pop,98934105,85,the boy is mine,eternal sunshine,08/03/2024,173639,True,85,...,0.630,7,-5.854,0,0.0434,0.1570,0.000000,0.0732,0.447,97.998
2,Ariana Grande,pop,98934105,83,intro (end of the world),eternal sunshine,08/03/2024,92400,True,83,...,0.362,10,-9.480,1,0.0416,0.6700,0.000000,0.1760,0.385,84.726
3,Ariana Grande,pop,98934105,80,Save Your Tears (Remix) (with Ariana Grande) -...,After Hours (Deluxe),20/03/2020,191013,False,80,...,0.825,0,-4.645,1,0.0325,0.0215,0.000024,0.0936,0.593,118.091
4,Ariana Grande,pop,98934105,79,"yes, and?",eternal sunshine,08/03/2024,214994,True,79,...,0.775,1,-6.614,1,0.0548,0.1900,0.000065,0.1130,0.787,118.998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Outkast,"atl hip hop, dirty south rap, hip hop, old sch...",2769972,69,Roses,Speakerboxxx/The Love Below,2003,369573,True,69,...,0.708,0,-5.653,1,0.0545,0.0482,0.000018,0.1070,0.509,108.970
996,Outkast,"atl hip hop, dirty south rap, hip hop, old sch...",2769972,64,Rosa Parks,Aquemini,29/09/1998,324373,True,64,...,0.701,6,-7.198,0,0.2830,0.1050,0.000000,0.7610,0.251,103.890
997,Outkast,"atl hip hop, dirty south rap, hip hop, old sch...",2769972,60,B.O.B. - Bombs Over Baghdad,Stankonia,31/10/2000,304226,True,60,...,0.978,9,-5.324,1,0.0978,0.0555,0.000041,0.0260,0.653,153.809
998,Outkast,"atl hip hop, dirty south rap, hip hop, old sch...",2769972,59,Da Art of Storytellin' (Pt. 1),Aquemini,29/09/1998,222653,True,59,...,0.856,2,-5.833,1,0.2350,0.1070,0.000000,0.7450,0.402,103.773


In [7]:
#Ahora que se encuentra mas limpio veamos nuevamente con que datos vamos a trabajar
df.dtypes


artist_name           object
genres                object
followers              int64
artist_popularity      int64
track_name            object
album_name            object
release_date          object
duration_ms            int64
explicit                bool
track_popularity       int64
danceability         float64
energy               float64
key                    int64
loudness             float64
mode                   int64
speechiness          float64
acousticness         float64
instrumentalness     float64
liveness             float64
valence              float64
tempo                float64
dtype: object

In [8]:
df.head()

,artist_name,genres,followers,artist_popularity,track_name,album_name,release_date,duration_ms,explicit,track_popularity,...,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Ariana Grande,pop,98934105,89,we can't be friends (wait for your love),eternal sunshine,08/03/2024,228639,False,89,...,0.646,5,-8.334,1,0.0427,0.0615,0.000030,0.0740,0.295,115.842
1,Ariana Grande,pop,98934105,85,the boy is mine,eternal sunshine,08/03/2024,173639,True,85,...,0.630,7,-5.854,0,0.0434,0.1570,0.000000,0.0732,0.447,97.998
2,Ariana Grande,pop,98934105,83,intro (end of the world),eternal sunshine,08/03/2024,92400,True,83,...,0.362,10,-9.480,1,0.0416,0.6700,0.000000,0.1760,0.385,84.726
3,Ariana Grande,pop,98934105,80,Save Your Tears (Remix) (with Ariana Grande) -...,After Hours (Deluxe),20/03/2020,191013,False,80,...,0.825,0,-4.645,1,0.0325,0.0215,0.000024,0.0936,0.593,118.091
4,Ariana Grande,pop,98934105,79,"yes, and?",eternal sunshine,08/03/2024,214994,True,79,...,0.775,1,-6.614,1,0.0548,0.1900,0.000065,0.1130,0.787,118.998


In [10]:
# Que release_date sea tipo object nos impide trabajar con fechas
df['release_date'] = pd.to_datetime(df['release_date'], format='%d/%m/Y')
df['release_date'] 

ValueError: time data "08/03/2024" doesn't match format "%d/%m/Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.